In [2]:
:dep lambdaworks-stark = { path = "." }
:dep lambdaworks-math = { git = "https://github.com/lambdaclass/lambdaworks", rev = "a17b951" }
:dep lambdaworks-crypto = { git = "https://github.com/lambdaclass/lambdaworks", rev = "a17b951" }
:dep once_cell

In [4]:
use std::ops::Range;

use lambdaworks_math::field::fields::{
    fft_friendly::stark_252_prime_field::Stark252PrimeField as F,
    u64_prime_field::{F17, FE17},
};
use lambdaworks_stark::{
    cairo::{
        air::{
            generate_cairo_proof, verify_cairo_proof, MemorySegment, MemorySegmentMap,
            PublicInputs, FRAME_DST_ADDR, FRAME_OP0_ADDR, FRAME_OP1_ADDR, FRAME_PC,
        },
        cairo_layout::CairoLayout,
        execution_trace::build_main_trace,
        runner::run::{
            cairo0_program_path, cairo1_program_path, generate_prover_args, run_program,
            CairoVersion,
        },
    },
    starks::{
        example::{
            dummy_air::{self, DummyAIR},
            fibonacci_2_columns::{self, Fibonacci2ColsAIR},
            fibonacci_rap::{fibonacci_rap_trace, FibonacciRAP, FibonacciRAPPublicInputs},
            quadratic_air::{self, QuadraticAIR, QuadraticPublicInputs},
            simple_fibonacci::{self, FibonacciAIR, FibonacciPublicInputs},
        },
        proof::options::{ProofOptions, SecurityLevel},
        prover::prove,
        trace::TraceTable,
        verifier::verify,
    },
    FE,
};

use lambdaworks_crypto::fiat_shamir::transcript::Transcript;
use lambdaworks_math::field::{element::FieldElement, traits::IsFFTField};

use lambdaworks_stark::starks::{
    constraints::boundary::{BoundaryConstraint, BoundaryConstraints},
    context::AirContext,
    frame::Frame,
    proof::options::ProofOptions,
    trace::TraceTable,
    traits::AIR,
};
use once_cell::sync::Lazy;
use std::sync::Mutex;

In [152]:
use std::sync::atomic::{AtomicUsize, Ordering::SeqCst};

In [213]:
static N_COLS: AtomicUsize = AtomicUsize::new(0);

pub fn change_n_cols(new_value: usize) -> usize {
    N_COLS.store(new_value, SeqCst);
    new_value
}

pub fn get_n_cols() -> usize {
    N_COLS.load(SeqCst)
}


#[derive(Clone)]
pub struct PellAIR<F>
where
    F: IsFFTField,
{
    context: AirContext,
    trace_length: usize,
    pub_inputs: PellPublicInputs<F>,
    n_columns: usize,
}

#[derive(Clone, Debug)]
pub struct PellPublicInputs<F>
where
    F: IsFFTField,
{
    pub a0: FieldElement<F>,
    pub a1: FieldElement<F>,
}

impl<F> AIR for PellAIR<F>
where
    F: IsFFTField,
{
    type Field = F;
    type RAPChallenges = ();
    type PublicInputs = PellPublicInputs<Self::Field>;

    fn new(
        trace_length: usize,
        pub_inputs: &Self::PublicInputs,
        proof_options: &ProofOptions,
    ) -> Self {
        let n_columns = get_n_cols();
        println!("La cantidad de columnas es {}", n_columns);
        let transition_offsets = if n_columns > 1 {vec![0, 1]} else {vec![0, 1, 2]};
        let transition_exemptions = if n_columns == 1 {
            vec![2]
        } else {
            vec![1, 1].into_iter().chain(std::iter::repeat(0).take(n_columns - 2)).collect()
        };
        let context = AirContext {
            proof_options: proof_options.clone(),
            trace_columns: n_columns,
            transition_degrees: vec![1; n_columns], // usamos la misma transicion en cada columna. 
            transition_offsets: transition_offsets,
            num_transition_constraints: n_columns,
            transition_exemptions: transition_exemptions,
            num_transition_exemptions: 1,
        };

        Self {
            pub_inputs: pub_inputs.clone(),
            context,
            trace_length,
            n_columns,
        }
    }

    fn composition_poly_degree_bound(&self) -> usize {
        self.trace_length()
    }

    fn build_auxiliary_trace(
        &self,
        _main_trace: &TraceTable<Self::Field>,
        _rap_challenges: &Self::RAPChallenges,
    ) -> TraceTable<Self::Field> {
        TraceTable::empty()
    }

    fn build_rap_challenges<T: Transcript>(&self, _transcript: &mut T) -> Self::RAPChallenges {}

    fn compute_transition(
        &self,
        frame: &Frame<Self::Field>,
        _rap_challenges: &Self::RAPChallenges,
    ) -> Vec<FieldElement<Self::Field>> {
        let first_row = frame.get_row(0);
        let second_row = frame.get_row(1);
        if self.n_columns > 1 {
            let mut transitions = vec![];
            // La primera restriccion es tal que s_{0, i+1} = 2 * s_{ncols - 1, i} + s_{ncols - 2, i}.
            transitions.push(second_row[0].clone() - FieldElement::<F>::from(2) * first_row[self.n_columns - 1].clone() - first_row[(2 * self.n_columns - 2) % self.n_columns].clone());
            // La segunda restriccion tambien usa la fila anterior siempre. 
            transitions.push(second_row[1].clone() - FieldElement::<F>::from(2) * second_row[0].clone() - first_row[self.n_columns - 1].clone());
            // Las demas transiciones son sobre la misma fila pero de las anteriores dos columnas.
            for i in 2..self.n_columns {
                transitions.push(second_row[i].clone() - FieldElement::<F>::from(2) * second_row[i-1].clone() - second_row[i-2].clone());
            }
            return transitions;
        }
        
        let third_row = frame.get_row(2);

        vec![third_row[0].clone() - FieldElement::<F>::from(2) * second_row[0].clone() - first_row[0].clone()]
    }

    fn boundary_constraints(
        &self,
        _rap_challenges: &Self::RAPChallenges,
    ) -> BoundaryConstraints<Self::Field> {
        let a0 = BoundaryConstraint::new(0, 0, self.pub_inputs.a0.clone());
        // si se hace con mas de una columna, la segunda restriccion es el primer
        // valor de la segunda columna.
        let a1 = if self.n_columns > 1 { 
            BoundaryConstraint::new(1, 0, self.pub_inputs.a1.clone())
        } else { 
            BoundaryConstraint::new(0, 1, self.pub_inputs.a1.clone())
        };

        BoundaryConstraints::from_constraints(vec![a0, a1])
    }

    fn number_auxiliary_rap_columns(&self) -> usize {
        0
    }

    fn context(&self) -> &AirContext {
        &self.context
    }

    fn trace_length(&self) -> usize {
        self.trace_length
    }

    fn pub_inputs(&self) -> &Self::PublicInputs {
        &self.pub_inputs
    }
}



In [214]:
pub fn pell_trace<F: IsFFTField>(
    initial_values: [FieldElement<F>; 2],
    trace_length: usize,
    n_columns: usize,
) -> TraceTable<F> {
    let mut ret: Vec<Vec<FieldElement<F>>> = vec![vec![]; n_columns];
    ret[0].push(initial_values[0].clone());
    
    // si hay mas de una columna, es el primer elemento de la segunda columna.
    // Sino, en la unica columna que hay 
    ret[1 % n_columns].push(initial_values[1].clone());
    let mut a_n_1 = initial_values[1].clone();
    let mut a_n_2 = initial_values[0].clone();

    for i in 2..(trace_length * n_columns) {        
        let res = FieldElement::<F>::from(2) * a_n_1.clone() + a_n_2.clone();
        ret[i % n_columns].push(res.clone());
        a_n_2 = a_n_1;
        a_n_1 = res;
    }
    TraceTable::new_from_cols(&ret)
}

In [219]:
fn test_prove_tp3seq() {
    let n_cols = change_n_cols(1024);
    let trace_length = 1
    let trace = pell_trace([FE::from(1), FE::from(1)], trace_length, n_cols);

    let proof_options = ProofOptions::default_test_options();

    let pub_inputs = PellPublicInputs {
        a0: FE::one(),
        a1: FE::one(),
    };

    let proof = prove::<F, PellAIR<F>>(&trace, &pub_inputs, &proof_options).unwrap();
    assert!(verify::<F, PellAIR<F>>(
        &proof,
        &pub_inputs,
        &proof_options
    ));
}

In [220]:
test_prove_tp3seq()

La cantidad de columnas es 1024
La cantidad de columnas es 1024


()

In [226]:
for i in 0..10 {
    let n_cols = change_n_cols(usize::pow(2,i));
    let trace_length = usize::pow(2,10 - i);
}

()